In [1]:
import os
import pandas as pd

main_publisher = 'Merged'

script_dir = os.path.dirname(os.path.realpath('__file__'))
path = os.path.join(script_dir, '../Data/' + main_publisher + '/')
df_concepts = pd.read_csv(path + 'concepts.csv', sep = '|')
df_concepts.head()

,OER,Concept,PR
0,0,http://en.wikipedia.org/wiki/%2edo,0.000132
1,0,http://en.wikipedia.org/wiki/%2eus,0.000230
2,0,http://en.wikipedia.org/wiki/%c3%85re,0.000108
3,0,http://en.wikipedia.org/wiki/12-hour_clock,0.000328
4,0,http://en.wikipedia.org/wiki/1964_Idaho_Vandal...,0.000016


In [72]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import time

def get_classes(concept):
    classes = []
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    request = """
                    SELECT ?class WHERE {
            <http://dbpedia.org/resource/CONCEPT> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class
        }
    """
    request = request.replace('CONCEPT', concept)
    sparql.setQuery(request)
    done = False
    while not done : 
        try:
            ret = sparql.queryAndConvert()

            for r in ret["results"]["bindings"]:
                value = r["class"]["value"]
                if ("/yago/" not in value) and ("/umbel/" not in value) and (".wikidata." not in value):
                    classes.append({
                        "Concept" : concept,
                        "Class" : value
                    })
            done = True
            print(f'Concept {concept} done.')
        except Exception as e:
            print('Runtime Error : ', e)
            print('Sleeping for 10 seconds...')
            time.sleep(10)
            print('Sleep done !')
    return classes

In [73]:
y = df_concepts['Concept'].apply(lambda x : get_classes(x.split('/')[-1]))
y = [item for sublist in y for item in sublist]
df_classes = pd.DataFrame(y)
df_classes.head(20)

Concept %2edo done.
Concept %2eus done.
Concept %c3%85re done.
Concept 12-hour_clock done.
Concept 1964_Idaho_Vandals_football_team done.
Concept 2009_timeline_of_the_Somali_Civil_War done.
Concept A-side_and_B-side done.
Concept A_Song_of_Ice_and_Fire done.
Concept A_priori_and_a_posteriori done.
Concept Abolitionism_in_the_United_States done.
Concept Academic_term done.
Concept Activism done.
Concept Adaptation done.
Concept Administrative_divisions_of_Amur_Oblast done.
Concept Aesthetics done.
Concept Alan_Gow done.
Concept Also done.
Concept Altar done.
Concept Alternative_medicine done.
Concept Altruism done.
Concept America_(yacht) done.
Concept American_studies done.
Concept Angle done.
Concept Anniversary done.
Concept Anthem done.
Concept Anyway_(Martina_McBride_song) done.
Concept Archaeology done.
Concept Ask_price done.
Concept Attention done.
Concept Bad_debt done.
Concept Banknote done.
Concept Bar_examination done.
Concept Barbarian done.
Concept Bass_Library done.
Conce

In [66]:
df_classes.to_csv(path + "classes.csv", sep = '|')